MNIST 검증 tensorflow 1.x 버전

colab 에서 실행하므로 가장 상위에 %tensorflow_version 1.x 추가

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime


mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("")
print("train.num = ", mnist.train.num_examples, 
      ", test.num = ", mnist.test.num_examples, 
      ", validation.num = ", mnist.validation.num_examples) 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

#### shape 및 type(mnist) 확인

In [3]:
print("type(mnist) = ", type(mnist), 
      ", type(mnist.train.images) = ", type(mnist.train.images), 
      ", type(mnist.train.labels) = ", type(mnist.train.labels))

print("\ntrain image shape = ", np.shape(mnist.train.images))
print("train label shape = ", np.shape(mnist.train.labels))
print("test image shape = ", np.shape(mnist.test.images))
print("test label shape = ", np.shape(mnist.test.labels))

print("\ntrain image shape = ", mnist.train.images.shape)
print("test image shape = ", mnist.test.images.shape)
print("validation image shape = ", mnist.validation.images.shape)

type(mnist) =  <class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'> , type(mnist.train.images) =  <class 'numpy.ndarray'> , type(mnist.train.labels) =  <class 'numpy.ndarray'>

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)

train image shape =  (55000, 784)
test image shape =  (10000, 784)
validation image shape =  (5000, 784)


#### train data 정규화 및 label 의 one-hot encoding 확인

In [4]:
# 정규화 확인
print("length of mnist.train.images = ", len(mnist.train.images))

for index in range(len(mnist.train.images)):
    
    min_val = np.min(mnist.train.images[index])
    max_val = np.max(mnist.train.images[index])
    
    if min_val < 0.0:
        print("min value is ", min_val, ", index = ", index)
        break
    
    if max_val > 1.0:
        print("max value is ", max_val, ", index = ", index)
        break
    
print("")
print(mnist.train.images[0])  # 정규화 확인을 위한 테스트 출력

length of mnist.train.images =  55000

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.   

In [5]:
# one-hot encoding 확인
print("length of mnist.train.images = ", len(mnist.train.labels))

for index in range(len(mnist.train.labels)):
    
    min_val = np.min(mnist.train.labels[index])
    max_val = np.max(mnist.train.labels[index])
    
    if min_val < 0.0:
        print("min value is ", min_val, ", index = ", index)
        break
    
    if max_val > 1.0:
        print("max value is ", max_val, ", index = ", index)
        break
    
print("")
print(mnist.train.labels[0])  # one-hot encoding 확인을 위한 테스트 출력

length of mnist.train.images =  55000

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


#### Hyper-Parameter 설정

In [0]:
# 입력노드, 은닉노드, 출력노드, 학습율, 반복횟수, 배치 개수 등 설정
learning_rate = 0.1  # 학습율
epochs = 50            # 반복횟수
batch_size = 100      # 한번에 입력으로 주어지는 MNIST 개수

input_nodes = 784     # 입력노드 개수
hidden_nodes = 100    # 은닉노드 개수
output_nodes = 10     # 출력노드 개수

#### 입력과 출력을 위한 플레이스홀더 정의

In [0]:
# 입력과 출력을 위한 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, input_nodes])  
T = tf.placeholder(tf.float32, [None, output_nodes])  

#### 가중치, 바이어스 정의

In [0]:
W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes]))  # 은닉층 가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_nodes]))               # 은닉층 바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes])) # 출력층 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))               # 출력층 바이어스 노드

In [0]:
Z2 = tf.matmul(X, W2) + b2    # 선형회귀 선형회귀 값 Z2
A2 = tf.nn.relu(Z2)           # 은닉층 출력 값 A2, sigmoid 대신 relu 사용

# 출력층 선형회귀  값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2, W3) + b3   

y = A3 = tf.nn.softmax(Z3)

In [0]:
loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T) )

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [0]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal( tf.argmax(A3, 1), tf.argmax(T, 1) )

# batch_size X 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

# index list 출력
accuracy_index = tf.cast(predicted_val, dtype=tf.float32)

# 예측값 처리
predicted_list = tf.argmax(A3, 1)

In [12]:
# Type Check
print('type(predicted_val) = ', type(predicted_val),  ', type(accuracy) = ', type(accuracy))
print('type(accuracy_index) =', type(accuracy_index), ', type(predicted_list) = ', type(predicted_list))

type(predicted_val) =  <class 'tensorflow.python.framework.ops.Tensor'> , type(accuracy) =  <class 'tensorflow.python.framework.ops.Tensor'>
type(accuracy_index) = <class 'tensorflow.python.framework.ops.Tensor'> , type(predicted_list) =  <class 'tensorflow.python.framework.ops.Tensor'>


In [13]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
        
    start_time = datetime.now()
    
    for i in range(epochs):    # 50 번 반복수행
        
        total_batch = int(mnist.train.num_examples / batch_size)  # 55,000 / 100

        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now()
    
    print("")
    print("Elapsed Time => ", end_time-start_time)
    print("")
    
    # Accuracy 확인
    test_x_data = mnist.test.images    # 10000 X 784
    test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data, T: test_t_data})
    
    print('\ntype(accuracy_val) = ', type(accuracy_val), ',  Accuracy = ', accuracy_val)

epochs =  0 , step =  0 , loss_val =  99.914665
epochs =  0 , step =  100 , loss_val =  4.37576
epochs =  0 , step =  200 , loss_val =  3.4258022
epochs =  0 , step =  300 , loss_val =  1.965544
epochs =  0 , step =  400 , loss_val =  1.0973184
epochs =  0 , step =  500 , loss_val =  2.440867
epochs =  1 , step =  0 , loss_val =  1.0210538
epochs =  1 , step =  100 , loss_val =  0.9440068
epochs =  1 , step =  200 , loss_val =  0.9870211
epochs =  1 , step =  300 , loss_val =  1.4072841
epochs =  1 , step =  400 , loss_val =  1.418917
epochs =  1 , step =  500 , loss_val =  1.3155763
epochs =  2 , step =  0 , loss_val =  1.1492896
epochs =  2 , step =  100 , loss_val =  0.81792784
epochs =  2 , step =  200 , loss_val =  0.4325835
epochs =  2 , step =  300 , loss_val =  0.37836304
epochs =  2 , step =  400 , loss_val =  0.9900483
epochs =  2 , step =  500 , loss_val =  0.49581987
epochs =  3 , step =  0 , loss_val =  0.6082109
epochs =  3 , step =  100 , loss_val =  0.50882626
epochs = 